In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import statsmodels.api as sm

C:\Users\chait\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
mat = pd.read_csv("C:/Users/chait/Documents/KSSEM/Final Yr Project/student-mat.csv", sep=';')
por = pd.read_csv("C:/Users/chait/Documents/KSSEM/Final Yr Project/student-por.csv", sep=';')

In [3]:
df = pd.concat([mat,por])

In [4]:
df.columns = ['school','sex','age','address','family_size','parents_status','mother_education','father_education',
           'mother_job','father_job','reason','guardian','commute_time','study_time','failures','school_support',
          'family_support','paid_classes','activities','nursery','desire_higher_edu','internet','romantic','family_quality',
          'free_time','go_out','weekday_alcohol_usage','weekend_alcohol_usage','health','absences','period1_score','period2_score','final_score']


In [5]:
df['final_grade'] = 'na'
df.loc[(df.final_score >= 15) & (df.final_score <= 20), 'final_grade'] = 'good' 
df.loc[(df.final_score >= 10) & (df.final_score <= 14), 'final_grade'] = 'fair' 
df.loc[(df.final_score >= 0) & (df.final_score <= 9), 'final_grade'] = 'poor' 
df.head(5)

school sex  age address family_size parents_status  mother_education  \
0     GP   F   18       U         GT3              A                 4   
1     GP   F   17       U         GT3              T                 1   
2     GP   F   15       U         LE3              T                 1   
3     GP   F   15       U         GT3              T                 4   
4     GP   F   16       U         GT3              T                 3   

   father_education mother_job father_job     ...     free_time go_out  \
0                 4    at_home    teacher     ...             3      4   
1                 1    at_home      other     ...             3      3   
2                 1    at_home      other     ...             3      2   
3                 2     health   services     ...             2      2   
4                 3      other      other     ...             3      2   

   weekday_alcohol_usage  weekend_alcohol_usage  health absences  \
0                      1                      1       3        6   
1                      1                      1       3        4   
2                      2                      3       3       10   
3                      1                      1       5        2   
4                      1                      2       5        4   

  period1_score period2_score final_score final_grade  
0             5             6           6        poor  
1             5             5           6        poor  
2             7             8          10        fair  
3            15            14          15        good  
4             6            10          10        fair  

[5 rows x 34 columns]

In [6]:
# create dataframe dfd for classification
dfd = df.copy()
dfd = dfd.drop([ 'final_score'], axis=1)

In [7]:
# label encode final_grade
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
dfd.final_grade = le.fit_transform(dfd.final_grade)

In [8]:
# dataset train_test_split
from sklearn.cross_validation import train_test_split
X = dfd.drop('final_grade',axis=1)
y = dfd.final_grade
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

C:\Users\chait\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
# get dummy varibles 
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [10]:
# see total number of features
len(list(X_train))

58

In [11]:
# find the optimal # of minimum samples leaf
from sklearn.tree import DecisionTreeClassifier

msl=[]
for i in range(1,58):
    tree = DecisionTreeClassifier(min_samples_leaf=i)
    t= tree.fit(X_train, y_train)
    ts=t.score(X_test, y_test)
    msl.append(ts)
msl = pd.Series(msl)
msl.where(msl==msl.max()).dropna()

42    0.872611
43    0.872611
44    0.872611
45    0.872611
46    0.872611
47    0.872611
48    0.872611
49    0.872611
50    0.872611
51    0.872611
52    0.872611
53    0.872611
54    0.872611
55    0.872611
56    0.872611
dtype: float64

In [12]:
# final model
tree = DecisionTreeClassifier(min_samples_leaf=17)
t= tree.fit(X_train, y_train)
print("Decisioin Tree Model Score" , ":" , t.score(X_train, y_train) , "," , 
      "Cross Validation Score" ,":" , t.score(X_test, y_test))

Decisioin Tree Model Score : 0.8986301369863013 , Cross Validation Score : 0.8535031847133758


In [13]:
# find a good # of estimators
from sklearn.ensemble import RandomForestClassifier

ne=[]
for i in range(1,58):
    forest = RandomForestClassifier()
    f = forest.fit(X_train, y_train)
    fs = f.score(X_test, y_test)
    ne.append(fs)
ne = pd.Series(ne)
ne.where(ne==ne.max()).dropna()

47    0.878981
dtype: float64

In [14]:
# find a good # of min_samples_leaf
from sklearn.ensemble import RandomForestClassifier
ne=[]
for i in range(1,58):
 forest = RandomForestClassifier(n_estimators=36, min_samples_leaf=i)
 f = forest.fit(X_train, y_train)
 fs = f.score(X_test, y_test)
 ne.append(fs)
ne = pd.Series(ne)
ne.where(ne==ne.max()).dropna()

4    0.875796
dtype: float64

In [15]:
# final model
tree = DecisionTreeClassifier(min_samples_leaf=17)
t= tree.fit(X_train, y_train)
print("Decisioin Tree Model Score" , ":" , t.score(X_train, y_train) , "," , 
      "Cross Validation Score" ,":" , t.score(X_test, y_test))

Decisioin Tree Model Score : 0.8986301369863013 , Cross Validation Score : 0.8535031847133758


In [16]:
from sklearn.svm import SVC
svc = SVC()
s= svc.fit(X_train, y_train)
print("SVC Model Score" , ":" , s.score(X_train, y_train) , "," ,
      "Cross Validation Score" ,":" , s.score(X_test, y_test))

SVC Model Score : 0.9342465753424658 , Cross Validation Score : 0.8821656050955414


In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial', solver='newton-cg',fit_intercept=True)

In [23]:
# find optimal # of features to use in the model
from sklearn.feature_selection import SelectKBest, chi2

ks=[]
for i in range(1,58):
    sk = SelectKBest(chi2, k=i)
    x_new = sk.fit_transform(X_train,y_train)
    x_new_test=sk.fit_transform(X_test,y_test)
    l = lr.fit(x_new, y_train)
    ll = l.score(x_new_test, y_test)
    ks.append(ll)  
    
ks = pd.Series(ks)
ks = ks.reindex(list(range(1,58)))
ks

1     0.891720
2     0.888535
3     0.901274
4     0.898089
5     0.894904
6     0.904459
7     0.347134
8     0.356688
9     0.366242
10    0.872611
11    0.885350
12    0.856688
13    0.305732
14    0.863057
15    0.885350
16    0.878981
17    0.869427
18    0.222930
19    0.222930
20    0.222930
21    0.222930
22    0.222930
23    0.222930
24    0.222930
25    0.222930
26    0.222930
27    0.222930
28    0.222930
29    0.840764
30    0.834395
31    0.824841
32    0.691083
33    0.831210
34    0.831210
35    0.872611
36    0.866242
37    0.222930
38    0.222930
39    0.222930
40    0.222930
41    0.222930
42    0.222930
43    0.222930
44    0.222930
45    0.222930
46    0.222930
47    0.222930
48    0.222930
49    0.222930
50    0.222930
51    0.222930
52    0.222930
53    0.222930
54    0.222930
55    0.222930
56    0.866242
57         NaN
dtype: float64

In [20]:
plt.figure(figsize=(10,5))
ks.plot.line()
plt.title('Feature Selction', fontsize=20)
plt.xlabel('Number of Feature Used', fontsize=16)
plt.ylabel('Prediction Accuracy', fontsize=16)

Text(0,0.5,'Prediction Accuracy')

In [21]:
ks.where(ks==ks.max()).dropna()

6    0.904459
dtype: float64

In [24]:
# final model
sk = SelectKBest(chi2, k=8)
x_new = sk.fit_transform(X_train,y_train)
x_new_test=sk.fit_transform(X_test,y_test)
lr = lr.fit(x_new, y_train)
print("Logistic Regression Model Score" , ":" , lr.score(x_new, y_train) , "," ,
      "Cross Validation Score" ,":" , lr.score(x_new_test, y_test))

Logistic Regression Model Score : 0.8876712328767123 , Cross Validation Score : 0.3471337579617834
